In [ ]:
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import json

# Spotify credentials (replace with your values)
client_id = "d9b707a45cde4336ac30cf6fa156ff08"
client_secret = "e2f9a8ae1e274a49ab10cda4351f49a3"

# Auth
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

# Load dataset
df = pd.read_csv("final_music_data.csv")
unique_songs = df[['name', 'artists']].drop_duplicates()

# Convert to list of tuples
songs_list = list(unique_songs.itertuples(index=False, name=None))

# For storing results
image_lookup = {}

# Fetch function
def fetch_art(song_tuple):
    name, artist = song_tuple
    query = f"track:{name} artist:{artist}"
    try:
        results = sp.search(q=query, type='track', limit=1)
        items = results['tracks']['items']
        if items:
            return (song_tuple, items[0]['album']['images'][0]['url'])
    except:
        pass
    return (song_tuple, "")

# Run in threads
print("⚡ Fetching with threads...")

with ThreadPoolExecutor(max_workers=12) as executor:
    futures = [executor.submit(fetch_art, song) for song in songs_list]
    for future in tqdm(as_completed(futures), total=len(futures)):
        key, url = future.result()
        image_lookup[key] = url

# Apply to full DataFrame
df['image_url'] = df.apply(lambda row: image_lookup.get((row['name'], row['artists']), ""), axis=1)

# Save
df.to_csv("final_music_data_with_spotify_images_fast.csv", index=False)
print("✅ Done! Saved as 'final_music_data_with_spotify_images_fast.csv'")